In [1]:
# import dependencies
import os
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# get dataset
dataset_path = os.path.join('Data', 'flights.csv')
dataset = pd.read_csv(dataset_path, low_memory=False)

In [3]:
# convert to dataframe
flight_df = pd.DataFrame(dataset)
flight_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# add delayed column to show if flight was delayed (1) or not (0)
flight_df['Delayed'] = ''

In [5]:
# set Delayed column if flight was delayed
for index, row in flight_df.head(30).iterrows():
    # TODO: check if other columns are also null
    if(pd.isnull(row['AIR_SYSTEM_DELAY']) == False):
        flight_df.set_value(index, 'Delayed', 1)

In [ ]:
#subsetting the dataframe to only include fligths delayed more than 60 minutes
late_flights = flight_df.loc[flight_df['ARRIVAL_DELAY'] >= 60]
late_flights = late_flights.drop(['FLIGHT_NUMBER', 'TAIL_NUMBER', 'DIVERTED', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY'], axis=1)
late_flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,...,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,CANCELLED,CANCELLATION_REASON,WEATHER_DELAY,Delayed
52,2015,1,1,4,B6,SJU,MCO,400,535.0,95.0,...,1189,727.0,3.0,605,730.0,85.0,0,NaN,0.0,
55,2015,1,1,4,B6,SJU,BDL,438,550.0,72.0,...,1666,902.0,6.0,739,908.0,89.0,0,NaN,0.0,
70,2015,1,1,4,AA,DFW,MIA,515,703.0,108.0,...,1121,1031.0,7.0,856,1038.0,102.0,0,NaN,102.0,
73,2015,1,1,4,US,PDX,PHX,520,620.0,60.0,...,1009,945.0,5.0,850,950.0,60.0,0,NaN,0.0,
86,2015,1,1,4,AA,DEN,DFW,530,623.0,53.0,...,641,931.0,10.0,835,941.0,66.0,0,NaN,0.0,


In [ ]:
for row in late_flights:
    #late_flights['DATE'] = pd.to_datetime(late_flights[['YEAR', 'MONTH', 'DAY']])
    late_flights['DATE'] = late_flights['YEAR'].astype(str) + late_flights['MONTH'].astype(str) + late_flights['DAY'].astype(str)

In [ ]:
late_flights.head()

In [ ]:

plt.scatter(late_flights['ARRIVAL_DELAY'], late_flights['DATE'])
plt.show()    
    
    
